In [1]:
import numpy as np
import pandas as pd
import os
import datetime
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import warnings
warnings.simplefilter("ignore")

In [4]:
import data_feeder

In [6]:
from sklearn.linear_model import LinearRegression

## Main Class

In [8]:
class PearsonBot:
    def __init__(self, timeframe: int = 30, min_linreg: int = 3) -> object:
        self.timeframe = timeframe
        self.min_linreg = min_linreg
        self.df_timeframe = pd.DataFrame()
        self.counter = 0
        self.m = np.nan
        self.c = np.nan
        self.process = True
        self.std = np.nan
        self.temp_flag = True
    
    def flush_counter(self):
        self.counter = 0
    
    def flush_df(self):
        self.df_timeframe = pd.DataFrame()
    
    def flush_coeffs(self):
        self.m = np.nan
        self.c = np.nan
    
    def flush_all(self):
        self.flush_df()
        self.flush_counter()
        self.flush_coeffs()
    
    def calc_hl2(self) -> None:
        self.df_timeframe['hl2'] = ((self.df_timeframe['High'] - self.df_timeframe['Low']) / 2) + self.df_timeframe['Low']
    
    def calc_coeffs(self, df):
        lr = LinearRegression()
        lr.fit(df[['index']], df['hl2'])
        self.m = lr.coef_
        self.c = lr.intercept_
    
    def lin_reg_fn(self, x):
        return (self.m * x) + self.c
    
    def calc_lr(self):
        self.calc_coeffs()
        self.df_timeframe['lin_reg'] = self.df_timeframe['index'].apply(self.lin_reg_fn)

    def on_tick(self, df: pd.DataFrame):
        if self.counter <= self.timeframe:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            self.df_timeframe = pd.concat([self.df_timeframe, df])
        else:
            if self.process:
                self.calc_hl2()
                self.df_timeframe = self.df_timeframe.reset_index().drop(columns='index').reset_index()
                self.calc_lr()
                self.std = self.df_timeframe['hl2'].std()
                self.lin_reg_height = (self.df_timeframe['lin_reg'].max() - self.df_timeframe['lin_reg'].min())
                self.process = False
            else:
                print('monitoring...')
                self.flush_all()
                self.temp_flag = False      
    
    def display(self):
        self.df_timeframe[['hl2','lin_reg']].plot(figsize=(15,15))
    
    def main(self):
        for df in tqdm(data_feeder.get_ticks()):
            if self.temp_flag:
                self.on_tick()
            else:
                break
        
        self.display()

In [9]:
pb = PearsonBot()

In [10]:
pb.main()

TypeError: get_ticks() missing 1 required positional argument: 'df'